In [ ]:
%pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Kiểm tra xem TensorFlow có nhận GPU không (để train nhanh hơn)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
class AudioH5Generator(Sequence):
    def __init__(self, h5_path, list_IDs, batch_size=16, shuffle=True):
        """
        h5_path: Đường dẫn tới file data_2d.h5
        list_IDs: Danh sách các index (số thứ tự) mẫu sẽ dùng (để chia train/val)
        batch_size: Kích thước lô (giảm xuống nếu vẫn bị tràn VRAM GPU)
        """
        self.h5_path = h5_path
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        # Tính số lượng batch trong 1 epoch
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # Lấy ra danh sách index cho batch hiện tại
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Gọi hàm lấy dữ liệu thực tế
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        # Xáo trộn dữ liệu sau mỗi epoch
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # Mở file H5
        with h5py.File(self.h5_path, 'r') as f:
            # Lấy Dataset
            X_dset = f['X_train']
            Y_dset = f['Y_train']

            # Cách 1: Dùng List Comprehension (An toàn nhất)
            # Duyệt qua từng ID trong batch và lấy dữ liệu từng cái một
            # h5py hỗ trợ rất tốt việc lấy đơn lẻ f['X_train'][10]
            X_batch = [X_dset[int(ID)] for ID in list_IDs_temp]
            y_batch = [Y_dset[int(ID)] for ID in list_IDs_temp]

        # Chuyển list thành numpy array để trả về cho Model
        return np.array(X_batch), np.array(y_batch)

In [ ]:
DATA_PATH = '../media_files/preprocessed_audio/data_2d.h5'
BATCH_SIZE = 16 # Nếu GPU yếu (như GTX 1650/1050), hãy giảm xuống 8 hoặc 4

# 1. Mở file để xem tổng số mẫu
with h5py.File(DATA_PATH, 'r') as f:
    total_samples = f['X_train'].shape[0]
    input_shape = f['X_train'].shape[1:] # (1024, 128, 1)
    print(f"Tổng số mẫu dữ liệu: {total_samples}")
    print(f"Kích thước đầu vào: {input_shape}")

# 2. Tạo danh sách ID và chia Train/Val (80% Train, 20% Val)
all_indices = np.arange(total_samples)
train_ids, val_ids = train_test_split(all_indices, test_size=0.2, random_state=42)

print(f"Số mẫu Train: {len(train_ids)}")
print(f"Số mẫu Val: {len(val_ids)}")

# 3. Khởi tạo Generators
training_generator = AudioH5Generator(DATA_PATH, train_ids, batch_size=BATCH_SIZE)
validation_generator = AudioH5Generator(DATA_PATH, val_ids, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    # --- ENCODER ---
    # Block 1
    c1 = Conv2D(16, (3, 3), padding='same', activation='relu')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(16, (3, 3), padding='same', activation='relu')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    # Block 2
    c2 = Conv2D(32, (3, 3), padding='same', activation='relu')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(32, (3, 3), padding='same', activation='relu')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Block 3
    c3 = Conv2D(64, (3, 3), padding='same', activation='relu')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(64, (3, 3), padding='same', activation='relu')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    
    # Block 4
    c4 = Conv2D(128, (3, 3), padding='same', activation='relu')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(128, (3, 3), padding='same', activation='relu')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # --- BOTTLENECK ---
    b = Conv2D(256, (3, 3), padding='same', activation='relu')(p4)
    b = BatchNormalization()(b)
    b = Conv2D(256, (3, 3), padding='same', activation='relu')(b)

    # --- DECODER ---
    # Block 4 Up
    u1 = UpSampling2D((2, 2))(b)
    u1 = Concatenate()([u1, c4])
    c5 = Conv2D(128, (3, 3), padding='same', activation='relu')(u1)
    c5 = BatchNormalization()(c5)
    
    # Block 3 Up
    u2 = UpSampling2D((2, 2))(c5)
    u2 = Concatenate()([u2, c3])
    c6 = Conv2D(64, (3, 3), padding='same', activation='relu')(u2)
    c6 = BatchNormalization()(c6)

    # Block 2 Up
    u3 = UpSampling2D((2, 2))(c6)
    u3 = Concatenate()([u3, c2])
    c7 = Conv2D(32, (3, 3), padding='same', activation='relu')(u3)
    c7 = BatchNormalization()(c7)
    
    # Block 1 Up
    u4 = UpSampling2D((2, 2))(c7)
    u4 = Concatenate()([u4, c1])
    c8 = Conv2D(16, (3, 3), padding='same', activation='relu')(u4)
    c8 = BatchNormalization()(c8)

    # Output (Sigmoid cho mask 0-1)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c8)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

model = build_unet(input_shape)
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
model.summary()

In [ ]:
# Checkpoint: Chỉ lưu model khi validation loss giảm (Model tốt nhất)
checkpoint = ModelCheckpoint(
    'best_unet_vocal.keras', # Đuôi .keras là chuẩn mới của TensorFlow
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    mode='min'
)

# ReduceLR: Giảm learning rate nếu loss không giảm sau 3 epochs (giúp hội tụ sâu hơn)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=3, 
    min_lr=1e-6, 
    verbose=1
)

# EarlyStopping: Dừng train nếu loss không giảm sau 5 epochs (tránh tốn điện)
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

# BẮT ĐẦU TRAIN
history = model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=50, # Đặt nhiều, EarlyStopping sẽ lo việc dừng sớm
    callbacks=[checkpoint, reduce_lr, early_stop]
)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Curve')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Curve')
plt.legend()
plt.show()

In [ ]:
# Lấy 1 batch từ tập validation để test
X_test, y_test = validation_generator[0]

# Dự đoán
y_pred = model.predict(X_test)

# Vẽ 3 hình: Input Mix, True Mask, Predicted Mask
def visualize_sample(index):
    plt.figure(figsize=(15, 5))
    
    # 1. Input Spectrogram (Mix)
    plt.subplot(1, 3, 1)
    # Xoay ngược trục Y (origin='lower') để tần số thấp ở dưới
    plt.imshow(X_test[index, :, :, 0], aspect='auto', origin='lower', cmap='magma')
    plt.title("Input Mixture Spectrogram")
    
    # 2. True Mask (Vocal gốc)
    plt.subplot(1, 3, 2)
    plt.imshow(y_test[index, :, :, 0], aspect='auto', origin='lower', cmap='gray')
    plt.title("Ground Truth Vocal Mask")
    
    # 3. Predicted Mask (Model dự đoán)
    plt.subplot(1, 3, 3)
    plt.imshow(y_pred[index, :, :, 0], aspect='auto', origin='lower', cmap='gray')
    plt.title("Predicted Vocal Mask")
    
    plt.tight_layout()
    plt.show()

# Hiển thị kết quả của mẫu thứ 0 và mẫu thứ 5 trong batch
visualize_sample(0)
visualize_sample(5)